In [ ]:
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 6.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from arch import arch_model
import warnings
from datetime import datetime

# 데이터 로드 (CSV 파일로 변환된 데이터를 사용합니다)
file_path = '/content/산화리튬.csv'  # 이미지 파일 경로 (CSV 파일로 가정)
data = pd.read_csv(file_path)

# 날짜 형식으로 변환
data['년월'] = pd.to_datetime(data['년월'], format='%Y%m')

# 1987년 12월 이후 데이터 필터링
filtered_data = data[data['년월'] >= '1987-12-01']

# 필터링된 데이터 확인
if filtered_data.empty:
    raise ValueError("필터링된 데이터가 비어 있습니다. 조건을 확인하세요.")

# 수익률 계산
returns = filtered_data['수입금액'].pct_change().dropna()

# GARCH(1, 1) 모델 적용
garch_model = arch_model(returns, vol='Garch', p=1, q=1)

# 경고 무시
warnings.filterwarnings("ignore", category=UserWarning)

# GARCH 모델 피팅
garch_fit = garch_model.fit(disp='off')

# 미래 변동성 예측
forecast = garch_fit.forecast(horizon=5)
volatility_values = forecast.variance.values[-1]
print(forecast.variance[-1:],'\n')

# 변동성 값을 일일 변동성 수준으로 변환 (분산의 제곱근)
daily_volatility = [np.sqrt(v) for v in volatility_values]

# 변동성 수준에 따라 구분
volatility_levels = []

for vol in daily_volatility:
    if vol < 1:
        level = "낮은 변동성 (안정적인 상태)"
    elif vol < 2:
        level = "중간 변동성 (일반적인 상태)"
    elif vol < 4:
        level = "높은 변동성 (위험한 상태)"
    else:
        level = "매우 높은 변동성 (매우 위험한 상태)"
    volatility_levels.append(level)

# 결과 출력
for i, (vol, level) in enumerate(zip(daily_volatility, volatility_levels), 1):
    print(f"h.{i}: 일일 변동성 수준 = {vol:.2f}%, 변동성 수준 = {level}")

# 월간 변동성 계산
monthly_returns = filtered_data.set_index('년월')['수입금액'].resample('M').last().pct_change().dropna()

# NaN 값과 inf 값 제거
monthly_returns = monthly_returns.replace([np.inf, -np.inf], np.nan).dropna()

# GARCH(1, 1) 모델 적용 (월간)
garch_model_monthly = arch_model(monthly_returns, vol='Garch', p=1, q=1)

# GARCH 모델 피팅 (월간)
garch_fit_monthly = garch_model_monthly.fit(disp='off')

# 월간 변동성 예측
forecast_monthly = garch_fit_monthly.forecast(horizon=12)
volatility_values_monthly = forecast_monthly.variance.values[-1]

# 변동성 값을 월간 변동성 수준으로 변환 (분산의 제곱근)
monthly_volatility = [np.sqrt(v) for v in volatility_values_monthly]

# 변동성 수준에 따라 구분 (월간)
monthly_volatility_levels = []

for vol in monthly_volatility:
    if vol < 1:
        level = "낮은 변동성 (안정적인 상태)"
    elif vol < 2:
        level = "중간 변동성 (일반적인 상태)"
    elif vol < 4:
        level = "높은 변동성 (위험한 상태)"
    else:
        level = "매우 높은 변동성 (매우 위험한 상태)"
    monthly_volatility_levels.append(level)

# 결과 출력 (월간 변동성)
print("\n월간 변동성 수준:")
for i, (vol, level) in enumerate(zip(monthly_volatility, monthly_volatility_levels), 1):
    print(f"Month {i}: 월간 변동성 수준 = {vol:.2f}%, 변동성 수준 = {level}")


            h.1        h.2         h.3         h.4        h.5
436  159.223157  159.17418  159.125448  159.076958  159.02871 

h.1: 일일 변동성 수준 = 12.62%, 변동성 수준 = 매우 높은 변동성 (매우 위험한 상태)
h.2: 일일 변동성 수준 = 12.62%, 변동성 수준 = 매우 높은 변동성 (매우 위험한 상태)
h.3: 일일 변동성 수준 = 12.61%, 변동성 수준 = 매우 높은 변동성 (매우 위험한 상태)
h.4: 일일 변동성 수준 = 12.61%, 변동성 수준 = 매우 높은 변동성 (매우 위험한 상태)
h.5: 일일 변동성 수준 = 12.61%, 변동성 수준 = 매우 높은 변동성 (매우 위험한 상태)

월간 변동성 수준:
Month 1: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 2: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 3: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 4: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 5: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 6: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 7: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 8: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 9: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 10: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 변동성 (안정적인 상태)
Month 11: 월간 변동성 수준 = 0.38%, 변동성 수준 = 낮은 

In [ ]:
import numpy as np
import pandas as pd
from arch import arch_model
import warnings
from datetime import datetime

# 데이터 로드 (CSV 파일로 변환된 데이터를 사용합니다)
file_path = '/content/lithium_prices.csv'  # 이미지 파일 경로 (CSV 파일로 가정)
data = pd.read_csv(file_path)

# 날짜 형식으로 변환
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')



# 가격 데이터에서 수익률 계산
data['returns'] = data['price'].pct_change().dropna()
data = data.dropna(subset=['returns'])

# 수익률을 백분율로 변환
data['returns'] = data['returns'] * 100

# GARCH(1, 1) 모델 적용
garch_model = arch_model(data['returns'], vol='Garch', p=1, q=1)

# 경고 무시
warnings.filterwarnings("ignore", category=UserWarning)

# GARCH 모델 피팅
garch_fit = garch_model.fit(disp='off')

# 미래 변동성 예측
forecast = garch_fit.forecast(horizon=5)
volatility_values = forecast.variance.values[-1]
print(forecast.variance[-1:],'\n')

# 변동성 값을 일일 변동성 수준으로 변환 (분산의 제곱근)
daily_volatility = [np.sqrt(v) for v in volatility_values]

# 변동성 수준에 따라 구분
volatility_levels = []

for vol in daily_volatility:
    if vol < 1:
        level = "낮은 변동성 (안정적인 상태)"
    elif vol < 2:
        level = "중간 변동성 (일반적인 상태)"
    elif vol < 4:
        level = "높은 변동성 (위험한 상태)"
    else:
        level = "매우 높은 변동성 (매우 위험한 상태)"
    volatility_levels.append(level)

# 결과 출력
for i, (vol, level) in enumerate(zip(daily_volatility, volatility_levels), 1):
    print(f"h.{i}: 일일 변동성 수준 = {vol:.2f}%, 변동성 수준 = {level}")


           h.1       h.2     h.3       h.4       h.5
3684  1.025256  1.042271  1.0591  1.075744  1.092207 

h.1: 일일 변동성 수준 = 1.01%, 변동성 수준 = 중간 변동성 (일반적인 상태)
h.2: 일일 변동성 수준 = 1.02%, 변동성 수준 = 중간 변동성 (일반적인 상태)
h.3: 일일 변동성 수준 = 1.03%, 변동성 수준 = 중간 변동성 (일반적인 상태)
h.4: 일일 변동성 수준 = 1.04%, 변동성 수준 = 중간 변동성 (일반적인 상태)
h.5: 일일 변동성 수준 = 1.05%, 변동성 수준 = 중간 변동성 (일반적인 상태)


In [ ]:
import pandas as pd
import numpy as np
from arch import arch_model

# 데이터 불러오기 (이미 불러온 데이터를 사용)
data = pd.read_csv('/content/lithium_prices.csv')  # 파일 경로는 실제 파일 경로로 대체

# 'date'를 날짜 형식으로 변환
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

# 'date'를 인덱스로 설정
data.set_index('date', inplace=True)

# 일일 가격에서 월말 가격으로 리샘플링
monthly_prices = data['price'].resample('M').last()

# 월간 수익률 계산
monthly_returns = monthly_prices.pct_change().dropna() * 100

# GARCH(1, 1) 모델 적용
garch_model = arch_model(monthly_returns, vol='Garch', p=1, q=1)
garch_fitted = garch_model.fit(disp='off')

# 12개월치 변동성 예측
forecast = garch_fitted.forecast(horizon=12)
monthly_volatility = forecast.variance.iloc[-1] ** 0.5

# 결과 출력
print("12개월치 월간 변동성 예측:")
print(monthly_volatility, '\n')


# 변동성 수준 구분 함수
def volatility_level(vol):
    if vol < 1:
        return "낮은 변동성 (안정적인 상태)"
    elif 1 <= vol < 2:
        return "중간 변동성 (일반적인 상태)"
    else:
        return "높은 변동성 (불안정한 상태)"

# 변동성 수준 구분
volatility_levels = monthly_volatility.apply(volatility_level)

# 결과 출력
print("12개월치 월간 변동성 예측 및 수준 구분:")
for i, (vol, level) in enumerate(zip(monthly_volatility, volatility_levels)):
    print(f"Month {i+1}: 변동성 = {vol:.2f}%, 수준 = {level}")

12개월치 월간 변동성 예측:
h.01     5.925951
h.02     6.620848
h.03     7.249439
h.04     7.827713
h.05     8.366112
h.06     8.871897
h.07     9.350363
h.08     9.805510
h.09    10.240447
h.10    10.657649
h.11    11.059124
h.12    11.446525
Name: 2024-06-30 00:00:00, dtype: float64 

12개월치 월간 변동성 예측 및 수준 구분:
Month 1: 변동성 = 5.93%, 수준 = 높은 변동성 (불안정한 상태)
Month 2: 변동성 = 6.62%, 수준 = 높은 변동성 (불안정한 상태)
Month 3: 변동성 = 7.25%, 수준 = 높은 변동성 (불안정한 상태)
Month 4: 변동성 = 7.83%, 수준 = 높은 변동성 (불안정한 상태)
Month 5: 변동성 = 8.37%, 수준 = 높은 변동성 (불안정한 상태)
Month 6: 변동성 = 8.87%, 수준 = 높은 변동성 (불안정한 상태)
Month 7: 변동성 = 9.35%, 수준 = 높은 변동성 (불안정한 상태)
Month 8: 변동성 = 9.81%, 수준 = 높은 변동성 (불안정한 상태)
Month 9: 변동성 = 10.24%, 수준 = 높은 변동성 (불안정한 상태)
Month 10: 변동성 = 10.66%, 수준 = 높은 변동성 (불안정한 상태)
Month 11: 변동성 = 11.06%, 수준 = 높은 변동성 (불안정한 상태)
Month 12: 변동성 = 11.45%, 수준 = 높은 변동성 (불안정한 상태)
